In [4]:
import RAKE
%ls

 Il volume nell'unit… J Š HDD
 Numero di serie del volume: 7C24-9F9D

 Directory di J:\Downloads\NLP\kred

08/01/2023  16:18    <DIR>          .
05/01/2023  10:38    <DIR>          ..
05/01/2023  10:26               107 .gitignore
08/01/2023  14:58    <DIR>          .idea
05/01/2023  00:25    <DIR>          .vscode
05/01/2023  00:25                 0 __init__.py
05/01/2023  08:56    <DIR>          __pycache__
05/01/2023  08:56    <DIR>          base
05/01/2023  11:33         3.655.339 checkpoint.pt
05/01/2023  10:55             1.481 config.yaml
04/01/2023  17:14       131.711.608 data_dict_1672848868384.pkl
04/01/2023  17:52    <DIR>          datasets
05/01/2023  10:22        10.320.431 first100pairs_test_data.csv
05/01/2023  00:25           175.182 framework.PNG
05/01/2023  00:25            31.065 kred_example.ipynb
08/01/2023  16:18           101.899 KRED_train.ipynb
05/01/2023  00:25             9.149 KRED-Data_Preparation.ipynb
05/01/2023  13:01           434.464 KRED-Model_Train.

In [2]:
!git clone https://github.com/Mlawrence95/LinkedIn-Tech-Job-Data.git

Cloning into 'LinkedIn-Tech-Job-Data'...


In [4]:
%cd LinkedIn-Tech-Job-Data

/content/drive/MyDrive/kred/LinkedIn-Tech-Job-Data


In [3]:
!pip install --upgrade ipython
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     -------------------------------------- 775.8/775.8 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: ipython
    Found existing installation: ipython 8.7.0
    Uninstalling ipython-8.7.0:
      Successfully uninstalled ipython-8.7.0
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [ ]:
import gzip
import time
import torch
import os
from utils import *
from train_test import *
import argparse
from parse_config import ConfigParser


# Config Loading
parser = argparse.ArgumentParser(description='KRED')
parser.add_argument('-f')
parser.add_argument('-c', '--config', default="./config.json", type=str,
                    help='config file path (default: None)')
parser.add_argument('-r', '--resume', default=None, type=str,
                    help='path to latest checkpoint (default: None)')
parser.add_argument('-d', '--device', default=None, type=str,
                    help='indices of GPUs to enable (default: all)')

config = ConfigParser.from_args(parser)

data = load_pretrained_data_mind(config)


In [ ]:
if config['trainer']['training_type'] == "single_task":
    single_task_training(config, data)
else:
    multi_task_training(config, data)

model training
Training epoch 0/11 - 0.0
######
 Step: 0, 0.0 
######
######
 Step: 100, 0.7352941176470589 
######
all loss: tensor(67.9992, device='cuda:0', grad_fn=<AddBackward0>)
_train_epoch
auc socre: 0.6118746087795882
valid_socre
valid_scores
early_stopping
Saving checkpoint: out/saved/models/KRED/0105_110654/checkpoint-model-epoch1.pth ...
epoch % self.save_period
Training epoch 1/11 - 0.09090909090909091
######
 Step: 0, 0.0 
######
######
 Step: 100, 0.7352941176470589 
######
all loss: tensor(67.5875, device='cuda:0', grad_fn=<AddBackward0>)
_train_epoch
auc socre: 0.6117740784406757
valid_socre
valid_scores
early_stopping
Saving checkpoint: out/saved/models/KRED/0105_110654/checkpoint-model-epoch2.pth ...
epoch % self.save_period
Training epoch 2/11 - 0.18181818181818182
######
 Step: 0, 0.0 
######
######
 Step: 100, 0.7352941176470589 
######
all loss: tensor(66.4398, device='cuda:0', grad_fn=<AddBackward0>)
_train_epoch
auc socre: 0.6167082639081938
valid_socre
valid_sc

In [ ]:
test_data = data[-1]
testing(test_data, config)

auc score:0.6246956269465205
ndcg score:0.32927349361412417


In [13]:
!pip install -U pip setuptools wheel
!pip install spacy-transformers
!pip install pytextrank
!python -m spacy download en_core_web_trf
!python -m spacy download en_core_web_md
!python -m spacy download en_core_web_sm


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 53.5/53.5 kB 1.4 MB/s eta 0:00:00
     ---------------------------------------- 11.9/11.9 MB 8.5 MB/s eta 0:00:00
     -------------------------------------- 186.7/186.7 kB 5.7 MB/s eta 0:00:00
     ------------------------------------- 481.4/481.4 kB 10.0 MB/s eta 0:00:00
     ---------------------------------------- 48.9/48.9 kB ? eta 0:00:00
     ------------------------------------- 181.6/181.6 kB 10.7 MB/s eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 8.0 MB/s eta 0:00:00
     ---------------------------------------- 1.3/1.3 MB 8.1 MB/s eta 0:00:00
     ---------------------------------------- 56.8/56.8 kB ? eta 0:00:00
     ---------------------------------------- 96.8/96.8 kB 5.8 MB/s eta 0:00:00
     ---------------------------------------- 7.0/7.0 MB 7.2 MB/s eta 0:

In [231]:
#Employment type, Industries, Job function, Seniority level, company, company_id, context, date, description,
#education, location, months_experience, post_id, post_url, sal_high, sal_low, salary, title

# Full-time,Broadcast Media,Information Technology,Mid-Senior level,CyberCoders,21836.0,"{""@context"": ""http://schema.org"", ""@type"": ""JobPosting"", ""datePosted"": ""2021-07-09T17:39:28.000Z"", ""description"": ""Job Title: Senior Data Engineer&lt
import pandas as pd

df = pd.read_csv('datasets/SO-Indeed/jobs.csv')
records = df.to_dict(orient='records')

In [232]:
import spacy
import pywikibot
from spacy import displacy
from collections import Counter
import en_core_web_sm, en_core_web_trf, en_core_web_md
nlp = spacy.load("en_core_web_trf")

In [233]:
from spacy.pipeline.entity_linker import DEFAULT_NEL_MODEL
config = {
   "labels_discard": [],
   "n_sents": 0,
   "incl_prior": True,
   "incl_context": True,
   "model": DEFAULT_NEL_MODEL,
   "entity_vector_length": 64,
   "get_candidates": {'@misc': 'spacy.CandidateGenerator.v1'},
   "threshold": None,
}
doc = nlp(records[0]['description'])
nlp.add_pipe('entity_linker', config=config)

In [234]:
print([(X.text, X.label_) for X in doc.ents])

[('Alexandria', 'GPE'), ('VA', 'GPE'), ('$120k - $150k', 'MONEY'), ('AWS', 'ORG'), ('Alexandria', 'GPE'), ('VA', 'GPE'), ('DC', 'GPE'), ('at least 5 years', 'DATE'), ('Python', 'LANGUAGE'), ('AWS', 'ORG'), ('Google Cloud', 'ORG'), ('Spark', 'PRODUCT'), ('NoSQL', 'PRODUCT'), ('Machine Learning', 'ORG'), ('5+ years', 'DATE'), ('AWS', 'ORG'), ('Google Cloud', 'ORG'), ('Spark', 'PRODUCT'), ('401', 'PRODUCT'), ('all day', 'DATE'), ('every day', 'DATE'), ('today', 'DATE'), ('Shiv Warrier', 'PERSON'), ('U.S.', 'GPE'), ('CyberCoders, Inc', 'ORG'), ('the United States', 'GPE')]


In [248]:
from pywikibot.exceptions import NoPageError
graph = []

def get_wikidata(doc): #con trf e entity_linker
    # Iterate over the entities in the document
    for ent in doc.ents:
        # Get the entity label
        label = ent.label_

        # Get the entity text
        entity = ent.text

        # Search Wikidata for the entity
        site = pywikibot.Site("en", "wikipedia")
        page = pywikibot.Page(site, entity)

        # Get the Wikidata page for the entity
        try: graph.append({'entity': entity, "label": label, "WikidataId": page.data_item()})
        except NoPageError: None
        #wd_page = page.data_item()
        # Print the graph
        print(graph)
        # Add the entity to the graph
        return graph



In [236]:
print([d['WikidataId'].id  for d in graph if 'ETL' in d['entity']]) #wikidataID

[]


TextRank for most important sentence

In [255]:
import pytextrank

# example text
text = records[0]['description']

# load a spaCy model, depending on language, scale, etc. without NER
nlp = spacy.load("en_core_web_sm")
# source NER from the same pipeline package as the last component
#nlp.add_pipe('entity_linker', config=config)
# add PyTextRank to the spaCy pipeline
nlp.add_pipe("textrank")


In [259]:
doc = nlp(text)
l = []
def myFunc(e):
  return e['rank']

def construct_entity_info_title(doc):
    # examine the top-ranked phrases in the document
    for phrase in doc._.phrases:
        #d1 = nlp()
        l.append({ 'text': phrase.text, 'rank': phrase.rank})
        #print(phrase.text)
        #print(phrase.rank)

    l.sort(key=myFunc, reverse=True)

    return l[0] #first ranked

f = construct_entity_info_title(doc)

In [262]:
doc = nlp(f['text'])
# Get the entity text
entity = doc.text
# Search Wikidata for the entity
site = pywikibot.Site("en", "wikipedia")
page = pywikibot.Page(site, entity)
id = page.data_item()
print(id)

NoPageError: Page [[en:ETL experience]] doesn't exist.

non funziona fare prima il rank e poi wikidataID, provare contrario

In [192]:
import csv
#entity_info_title : [{"Label": "Weight gain", "Type": "C", "WikidataId": "Q3403879", "Confidence": 1.0, "OccurrenceOffsets": [45], "SurfaceForms": ["Gain Weight"]}]
jobs_info = [ 'post_id', 'industries','job_function', 'title', 'abstract', 'post_url', 'entity_info_title', 'entity_info_abstract']

new_dict = []
djob = {}
for job in records:
    djob["post_id"] = job['post_id']
    djob["industries"] = job['Industries']
    djob["job_function"] = job['Job function']
    djob["title"] = job['title']
    djob["abstract"] = job['description'].split('.')[0]
    djob["post_url"] = job['post_url']
    djob["entity_info_title"] = job['post_id']
    djob["entity_info_abstract"] = []
    new_dict.append(djob)


In [ ]:
with open('train_jobs.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = jobs_info)
    writer.writeheader()
    writer.writerows(new_dict)

#news | jobs
#id : post_id
#category: Industries
#subcategory : job function
#title: title
#abstract : requirements
#url : url
#entity_info_title : only one [{label, type, wikidataId, confidence, occurrence offsets, surface forms}]
#entity_info_abstract: all the first 3[{label, type, wikidataId, confidence, occurrence offsets, surface forms}]

In [85]:
!pip install spacy[transformers]
!pip install spacy[transformers,cuda111]

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 82.2/82.2 MB 2.4 MB/s eta 0:00:00


!pip install requests
!git clone https://gerrit.wikimedia.org/r/pywikibot/core.git
!cd core
!git submodule update --init
!python pwb.py script_name
!pip install pywikibot
!pip install "mwparserfromhell>=0.5.0"
!pip install "wikitextparser>=0.47.5"

import spacy
import pywikibot

# Load the en_core_web_md model
nlp = spacy.load("en_core_web_md")

# Define the text to process
text = "Barack Obama was born in Honolulu, Hawaii. He served as the 44th President of the United States from 2009 to 2017."

# Process the text using the model
doc = nlp(text)

# Create an empty graph
graph = {}

# Iterate over the entities in the document
for ent in doc.ents:
    # Get the entity label
    label = ent.label_

    # Get the entity text
    entity = ent.text

    # Search Wikidata for the entity
    site = pywikibot.Site("en", "wikipedia")
    page = pywikibot.Page(site, entity)
    print(entity, page.data_item())
    # Get the Wikidata page for the entity
    wd_page = page.data_item()
    # Add the entity to the graph
    #graph[entity] = {"label": label, "ID": wd_page}

# Print the graph
print(graph)
Barack Obama [[wikidata:Q76]]
Honolulu [[wikidata:Q18094]]
Hawaii [[wikidata:Q782]]